### 1. NLP2_1 https://www.hackerrank.com/challenges/detect-the-email-addresses/problem?isFullScreen=true

import re

N = int(input())

result = []

for _ in range(N):

    result.extend(re.findall(r'[\w.]+@[\w.]+\.\w{1,3}',input()))
    
print(';'.join(sorted(set(result))))



### 2. NLP2_2 https://www.hackerrank.com/challenges/detect-the-domain-name/problem?isFullScreen=true

import re

pattern = '(http|https)\\://(www.|ww2.|)([\w\\-\\.]+)(\\.[a-zA-Z]+)(/\\S*)?'

regex = re.compile (pattern)

s = set()

for i in range (int(input())):

    string = input ()
    
    iterator = regex.finditer(string)
    
    if iterator:
    
        for match in iterator:
        
            s.add(match.group(3)+match.group(4))
            
print(';'.join(t for t in sorted (s)))

### NLP2_3 (дз1): Реализовать stemming, lemmatization & BoW на следующем датасете: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK (Russian Toxic-abuse comments)


In [1]:
import pandas as pd
import numpy as np
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

In [2]:
# загружаем датасет
rta = pd.read_csv('Токсичные комментарии .zip')
rta.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [3]:
# проверяю токен в датафрейме по строчно
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(rta['comment'][0].lower()))

['верблюдов', '-', 'то', 'за', 'что', '?', 'дебилы', ',', 'бл', '...']


In [4]:
# лемматизатия
lemmatizer = WordNetLemmatizer()

dict_words = {}

for i_str in rta['comment']:
    for i_word in tokenizer.tokenize(i_str.lower()):
        word_lemmatizing = lemmatizer.lemmatize(i_word)
        if word_lemmatizing in dict_words.keys(): 
              dict_words[word_lemmatizing] += 1
        else:
              dict_words[word_lemmatizing] = 1
        
df = pd.DataFrame.from_dict(dict_words, orient='index')
df.sort_values(0, ascending = False).head(10)

,0
",",33932
.,26863
и,12684
в,11974
не,10301
-,7906
на,7003
что,5986
а,5008
?,4395


In [5]:
print('Количество слов после лемматизации', len(dict_words.values()))

Количество слов после лемматизации 68602


In [6]:
# стемминг

from nltk.stem.porter import *

stemmer = PorterStemmer()
stem = {}
for i_str in rta['comment']:
    for i_word in tokenizer.tokenize(i_str.lower()):
        stem_word = stemmer.stem(i_word)
        if stem_word in stem.keys(): 
            stem[stem_word] += 1
        else:
            stem[stem_word] = 1

stem_size = len(stem.values())

print("\nКоличество разных слов после стемминга : %.3f" % stem_size)


Количество разных слов после стемминга : 68602.000


### Bag Of Words (BOW)

In [7]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

stops = set(stopwords.words('russian'))

5#добавим в стоп слова еще слова
stops.add('бл')
stops.add('дебил')
stops.add('скот')
stops.add('тупорыл')

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?","-"]


In [8]:
#create a vocabulary list
vocab = set()
rta['filter'] = None
rta['flag'] = 0

# срез до 1000 записей
rta = rta[:1000]

for i_num, i_str in enumerate(rta['comment']):
    filtered_text = []
    for i_word in tokenizer.tokenize(i_str.lower()):
        stem_word = stemmer.stem(i_word)
        if stem_word not in stops and stem_word not in special_char: 
            filtered_text.append(stem_word)
        rta['filter'][i_num] = filtered_text
        if stem_word in stops:
            rta['flag'][i_num] = 1
        vocab.add(i_word)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in stops and w not in special_char: 
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
# vector1=vectorize(tokens1)
# print(vector1)
# vector2=vectorize(tokens2)
# print(vector2)

/var/folders/04/p9n_pr6j0n971vm_1r43l7400000gn/T/ipykernel_29779/584995612.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rta['filter'][i_num] = filtered_text
/var/folders/04/p9n_pr6j0n971vm_1r43l7400000gn/T/ipykernel_29779/584995612.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rta['flag'][i_num] = 1


['поработать', 'бою', 'живем', 'можете', 'развить', 'жулика', 'целое', 'собачья', 'обзывать', 'доска', 'пропускать', 'госперерворота', 'кричу', 'интеллектом', 'юрлицо', 'писькиному', 'всей', 'драк', 'жди', 'владимир', 'хуйня', 'распродаже', 'вон', 'покупается', 'скотоублюдушке', 'евгеническую', 'главой', 'обманывай', 'попавших', 'вводят', 'дружбу', 'думаю', 'классик', 'уполномоченного', 'нету', 'виднее', 'пафосную', 'поискал', 'повторить', 'бездействием', 'представь', 'сказать', 'окладе', 'александра', 'предъявил', 'подлец', 'знакомьтесь', 'полевой', 'лапшу', 'свободе', 'трудоемкий', 'полюдову', 'агрессивнее', 'сочную', 'простирался', 'бывшие', 'парадов', 'вчера', 'кавказцам', 'гражданам', 'согласно', 'топор', 'достанется', 'едет', 'отличие', 'штраф', 'потока', 'рассказал', 'соболева', 'албания', 'ячейку', 'напиток', 'взяли', 'заявлять', 'здоровье', 'вселенной', 'кусок', 'замедляет', 'мыслями', 'галочку', 'средство', 'сосиску', 'единственный', 'дичь', 'незащищёнными', 'научит', 'компью

In [9]:
rta

,comment,toxic,filter,flag
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[верблюдов, дебилы, ...]",1
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохлы, это, отдушина, затюканого, россиянина,...",1
2,Собаке - собачья смерть\n,1.0,"[собаке, собачья, смерть]",0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страницу, обнови, это, оскорбление, доказанны...",1
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[убедил, 6, страничный, пдф, скрипалей, отрави...",1
...,...,...,...,...
995,когда изобрели пулемёт максим думали войн уже ...,1.0,"[изобрели, пулемёт, максим, думали, войн, мань...",1
996,Стартер с аннигиляционной баркой(4000р) 8 крас...,0.0,"[стартер, аннигиляционной, баркой, (, 4000р, )...",1
997,Зачем Алекс Пыня залупается на людей умнее его...,1.0,"[алекс, пыня, залупается, людей, умнее, нравит...",1
998,"Конечно, я вас не поняла. Мы оба комментируем ...",0.0,"[поняла, оба, комментируем, omgh5lol, той, ист...",1


### NLP2_4 Реализовать классификатор токсичных комментариев tfidf на базе датасета https://www.kaggle.com/datasets/blackmoon/russian-language-toxic-comments

Дубликат файла: https://cloud.mail.ru/public/Z4L3/vB8GcgTtK

In [10]:
tox = rta[rta['flag'] == 1]['comment']
print('Токсичные комментарии', len(tox), 'шт\n', tox)


Токсичные комментарии 955 шт
 0                   Верблюдов-то за что? Дебилы, бл...\n
1      Хохлы, это отдушина затюканого россиянина, мол...
3      Страницу обнови, дебил. Это тоже не оскорблени...
4      тебя не убедил 6-страничный пдф в том, что Скр...
5      Для каких стан является эталоном современная с...
                             ...                        
995    когда изобрели пулемёт максим думали войн уже ...
996    Стартер с аннигиляционной баркой(4000р) 8 крас...
997    Зачем Алекс Пыня залупается на людей умнее его...
998    Конечно, я вас не поняла. Мы оба комментируем ...
999                            А вот у него мать пахала!
Name: comment, Length: 955, dtype: object


In [11]:
good = rta[rta['flag'] == 0]['comment']
print('Хорошие комментарии', len(good), 'шт\n', good)

Хорошие комментарии 45 шт
 2                              Собаке - собачья смерть\n
9                              Обосрался, сиди обтекай\n
28     https: pp.userapi.com c848520 v848520411 11627...
62                       Тамбовский волк тебе товарищ!\n
73                  Тупой-тупой-тупая, ти-ля-ля-лю-лю!\n
119                Тебе кольцо покупают , беги глупец!\n
148                         ви таки дурак, простите? )\n
194    Танкисты пидарасы танкисты хуесосы танкисты до...
204                  Пост-эталон малолетнего долбаёба.\n
206                           ряженые кизяки Извинись!\n
249                    разыгрывали 4000 рублей ( 2000)\n
253                  Эй, слющяй, чо оскорбляешь срязю?\n
302    добби свободен!!!!!!!!!!!!!!!!!!!! ох спасибо ...
314               Полно таких дебилов, незнающих тему.\n
326                   Тамбовская духовка тебе кулички!\n
354    Надеюсь кобре вырвали зубы. Иначе родители деб...
362    Хахаха. Автор пукнул мозгом... Продолжительные...
372 